### Threshold
1. Normalization
2. Histogram Equalization

In [1]:
import os
import cv2
import nibabel as nib
import numpy as np

from tqdm import tqdm
from matplotlib import pyplot as plt

%matplotlib inline

#### I. Simple Test

In [2]:
current_path = os.getcwd()

# sample_name = 'kku/kku_ori/sub-001.nii.gz'
# dest_path = os.path.join(current_path, "applied_combined8_under26.nii.gz")

sample_name = 'sample.nii.gz'
dest_path = os.path.join(current_path, "sample_result.nii.gz")

ori_nii_path = os.path.join(current_path, sample_name)

nii = nib.load(ori_nii_path)
nii_data = nii.get_fdata()

print(len(nii_data))

166


#### II. THRESHOLD Test: 1 case

In [ ]:
'''
BINARY THRESHOLD
- get the middle, max value
- threhold task using the values above

OTSU THRESHOLD
- apply threshold(with threshold binary + otsu)
''' 
max_val = np.max(nii_data)
mid_val = max_val/2
print(max_val, mid_val)
ret, threshold = cv2.threshold(nii_data, mid_val, max_val, cv2.THRESH_TRUNC)

# applied_img = nib.Nifti1Image(threshold, affine=nii.affine)
# nib.save(applied_img, dest_path)

'''
CLAHE
- normalize(0~1 or 0~255)
- type cast to unsigned int(8 bit)
- CLAHE task after the tasks above

ADAPTIVE THRESHOLD 
- X(Deprecated)
'''

### 3. apply bn threshold & normalize
norm_256 = np.max(threshold) / 255
img = threshold / norm_256
img = img.astype(np.uint8)
img[((np.where(img <= 30)))] = 0

### 1. normalize 0~1
# img = nii_data / np.max(nii_data) # normalize the data to 0 - 1
# img = 255 * img # Now scale by 255

### 2. normalize 0~255 & uint8
# norm_256 = np.max(nii_data) / 255
# img = nii_data / norm_256
# img = img.astype(np.uint8)

### check the values
# print(img.dtype, img.shape)
# print(np.min(img), np.max(img))
# print(len(img))

total_slices = []

## 1. apply CLAHE
clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8,8)) # 32, 32 > Poor, use 8, 8 or 16, 16
for slice in range(len(img)):
    applied_slice = clahe.apply(img[slice])
    total_slices.append(applied_slice)


## 2. apply adaptive threshold - deprecated
# for slice in range(len(img)):    ### in [ADAPTIVE_THRESH_MEAN_C, ADAPTIVE_THRESH_GAUSSIAN_C] cv2.THRESH_OTSU
#     applied_slice = cv2.adaptiveThreshold(img[slice], 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 55, 1)
#     total_slices.append(applied_slice)


## 3. apply otsu threshold - ??
# ret, threshold = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# applied_img = nib.Nifti1Image(threshold, affine=nii.affine)
# nib.save(applied_img, dest_path)


### common(1., 2. not 3.)
img = np.asarray(total_slices)

applied_img = nib.Nifti1Image(img, affine=nii.affine)
nib.save(applied_img, dest_path)

#### III. THRESHOLD or CLAHE: all cases
- KKU

In [ ]:
### apply clahe(histogram equalization)
def load_nifti(nii_path):
    nii = nib.load(nii_path)
    nii_data = nii.get_fdata()

    return nii, nii_data
    
    
def apply_clahe(nii_data):    
#     img = nii_data / np.max(nii_data) # normalize the data to 0 - 1
#     img = 255 * img # Now scale by 255
#     img = img.astype(np.uint8)

    norm_256 = np.max(nii_data) / 255
    img = nii_data / norm_256
    img = img.astype(np.uint8) 
    img[((np.where(img <= 30)))] = 0 # 약 1/10

    total_slices = []

    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8,8))
    for slice in range(len(img)):
        applied_slice = clahe.apply(img[slice])
        total_slices.append(applied_slice)

    img = np.asarray(total_slices)
    
    return img

    
### apply binary threshold    
def apply_bnThreshold(nii_data):
    max_val = np.max(nii_data)
    mid_val = max_val/2
    ret, threshold = cv2.threshold(nii_data, mid_val, max_val, cv2.THRESH_TRUNC)

    return threshold
    
    
current_path = os.getcwd() + '/kku'
ori_path = os.path.join(current_path, 'kku_ori')


# target_path = os.path.join(current_path, 'kku_clahe')
target_path = os.path.join(current_path, 'kku_combined') ### !!!!

not_exists = []

for i in tqdm(range(1, 187)):
    sub_name = f'sub-{str(i).zfill(3)}.nii.gz'
    ori_full_path = os.path.join(ori_path, sub_name)
    store_full_path = os.path.join(target_path, sub_name)
    
    try: 
        nii, nii_data = load_nifti(ori_full_path)
        img = apply_bnThreshold(nii_data) ### !!!!
        img = apply_clahe(img)
#         img = apply_clahe(nii_data)
    
        applied_img = nib.Nifti1Image(img, affine=nii.affine)
        nib.save(applied_img, store_full_path)
        
    except:
        not_exists.append(sub_name)
        
print(not_exists)
print(len(not_exists))
#         print(f"{sub_name} doesn't exist")
#     sub_list.append(str(i).zfill(3))

In [7]:
import SimpleITK as sitk
import nibabel as nib

input_img_raw = nib.load(ori_nii_path).get_fdata()
input_img = sitk.GetImageFromArray(input_img_raw.transpose(2,1,0))
input_img = sitk.Cast(input_img, sitk.sitkFloat32)

mask_img = sitk.OtsuThreshold(input_img, 0, 1, 200)

corrector = sitk.N4BiasFieldCorrectionImageFilter()
corrected_img = corrector.Execute(input_img, mask_img)

log_bias_field = sitk.Exp(corrector.GetLogBiasFieldAsImage(input_img))
corrected_img = input_img/log_bias_field

# bias_field_full_resolution = sitk.Exp(corrector.GetLogBiasFieldAsImage((input_img, mask_img))
# print(bias_field_full_resolution)
                                      
sitk.WriteImage(log_bias_field, os.path.join(os.getcwd(), "bias_mask.nii.gz"))
# sitk.WriteImage(corrected_img, os.path.join(os.getcwd(), "corrected.nii.gz"))
# print(log_bias_field)